# Homework 1 - Assignment 4

In [1]:
import numpy as np
import librosa
import os
import scipy as sp
%matplotlib inline
from tqdm import tqdm
import sklearn
import matplotlib.pyplot as plt
import IPython.display as ipd
from sklearn.mixture import BayesianGaussianMixture
import scipy.stats
import seaborn as sns
import csv
import tqdm   #to monitor loops during computation

In [9]:
def compute_metrics(gt_labels, predicted_labels):
    TP = np.sum(np.logical_and(predicted_labels == 1, gt_labels == 1))
    FP = np.sum(np.logical_and(predicted_labels == 1, gt_labels == 0))
    TN = np.sum(np.logical_and(predicted_labels == 0, gt_labels == 0))
    FN = np.sum(np.logical_and(predicted_labels == 0, gt_labels == 1))
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1_score = 2 * precision * recall / (precision + recall)
    print("Results : \n accuracy = {} \n precision = {} \n recall = {} \n F1 score = {}".format(
        accuracy, precision, recall, F1_score))

In [2]:
def compute_mfcc(audio, fs, n_mfcc):
    X = np.abs(librosa.stft(
    audio,
    window='hamming',
    n_fft = 1024,
    hop_length = 512,
    ))
    
    mel = librosa.filters.mel(
        sr = fs,
        n_fft = 1024,
        n_mels = 40,
        fmin = 133.33, #standard value
        fmax = 6853.8
    )
    
    melspectrogram = np.dot(mel, X)
    log_melspectrogram = np.log10(melspectrogram + 1e-16) 
    
    mfcc = sp.fftpack.dct(log_melspectrogram, axis=0, norm='ortho')[1:n_mfcc + 1]
    
    return mfcc

In [3]:
n_mfcc = 13

class Sound:
    
    def __init__(self, path, slice_file_name, fold, classID, class_name, startTime, endTime):
        self.slice_file_name = path +'/fold'+fold+'/'+slice_file_name
        self.fold = fold
        self.classID = classID
        self.class_name = class_name
        self.startTime = startTime
        self.endTime = endTime
        self.features = np.zeros(n_mfcc)
        
    def extract_features(self):
        audio, fs = librosa.load(self.slice_file_name, sr=None)
        mfcc = compute_mfcc(audio,fs, n_mfcc)
        self.features = np.mean(mfcc, axis=1)

In [4]:
## read csv

classes = ['air_conditioner',
           'car_horn',
           'children_playing',
           'dog_bark',
           'drilling',
           'engine_idling',
           'gun_shot',
           'jackhammer',
           'siren',
           'street_music'
          ]

path = 'UrbanSound/UrbanSound8K.csv'
soundList = []
with open(path, newline='') as soundcsv:
            reader = csv.DictReader(soundcsv)
            for row in reader:
                s = Sound('UrbanSound/audio', row['slice_file_name'], row['fold'], row['classID'], row['class'], row['start'], 
                          row['end'])
                s.extract_features()
                soundList.append(s)


/home/giovanni/programming/python/virtualenvs/cmls/lib/python3.8/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


In [5]:
def compute_training_set(test_folder):
    train_files = [s for s in soundList if s.fold!=test_folder]
    n_train_samples = len(train_files)
    train_features = np.zeros((n_train_samples, n_mfcc))
    train_labels = []
    
    for i,f in enumerate(train_files):
        train_features[i,:] = train_files[i].features
        train_labels.append(train_files[i].class_name)
        
    return [train_features, train_labels]

In [6]:
def compute_test_set(test_folder):
    test_files = [s for s in soundList if s.fold==test_folder]
    n_test_samples = len(test_files)
    test_features = np.zeros((n_test_samples, n_mfcc))
    
    for i,f in enumerate(test_files):
        test_features[i,:] = test_files[i].features
    
    return test_features

In [8]:
## compute training
k = 10

## fold cross validation
for i in np.arange(1,k):
    [train_features, train_labels] = compute_training_set(str(i))
    print("Size of train_features ",i,train_features.shape)
    print("Size of train_labels ",i,len(train_labels))
    test_features = compute_test_set(str(i))
    print("Size of test_features ",i,test_features.shape)
    model = sklearn.svm.SVC(C=1, kernel='rbf')
    model.fit(train_features, train_labels)
    predictions = model.predict(test_features)
    print("PREDICTIONS ON FOLD",i, predictions)

Size of train_features  1 (7859, 13)
Size of train_labels  1 7859
Size of test_features  1 (873, 13)
PREDICTIONS ON FOLD 1 ['dog_bark' 'dog_bark' 'dog_bark' 'children_playing' 'dog_bark'
 'engine_idling' 'street_music' 'air_conditioner' 'air_conditioner'
 'jackhammer' 'jackhammer' 'jackhammer' 'drilling' 'air_conditioner'
 'jackhammer' 'drilling' 'drilling' 'air_conditioner' 'air_conditioner'
 'drilling' 'drilling' 'air_conditioner' 'air_conditioner'
 'air_conditioner' 'drilling' 'drilling' 'drilling' 'drilling'
 'air_conditioner' 'air_conditioner' 'air_conditioner' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'children_playing'
 'children_playing' 'street_music' 'children_playing' 'children_playing'
 'children

PREDICTIONS ON FOLD 2 ['dog_bark' 'drilling' 'drilling' 'drilling' 'children_playing'
 'children_playing' 'children_playing' 'gun_shot' 'gun_shot' 'dog_bark'
 'dog_bark' 'dog_bark' 'siren' 'siren' 'siren' 'gun_shot' 'gun_shot'
 'dog_bark' 'dog_bark' 'dog_bark' 'dog_bark' 'dog_bark' 'dog_bark'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'street_music' 'street_music' 'street_music' 'street_music'
 'street_music' 'street_music' 'street_music' 'street_music'
 'street_music' 'street_music' 'street_music' 'street_music'
 'street_music' 'street_music' 'street_music' 'street_music'
 'street_music' 'street_music' 'street_music' 'street_music'
 'street_music' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'air_conditioner' 'street_music' 'engine_idling' 'jackhammer'
 'children_playing' 'children_playing' 'children_playing' 'engine_idling'
 'dog_bark' 'dog_bark' 'dog_bark' 'dog_bark' 'engine_idling'

PREDICTIONS ON FOLD 3 ['children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'drilling' 'children_playing'
 'drilling' 'drilling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'children_playing' 'air_conditioner' 'air_conditioner'
 'air_conditioner' 'gun_shot' 'dog_bark' 'dog_bark' 'dog_bark' 'dog_bark'
 'dog_bark' 'dog_bark' 'dog_bark' 'dog_bark' 'children_playing'
 'children_playing' 'children_playing' 'dog_bark' 'car_horn' 'car_horn'
 'car_horn' 'dog_bark' 'jackhammer' 'jackhammer' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playin

PREDICTIONS ON FOLD 4 ['children_playing' 'drilling' 'drilling' 'drilling' 'children_playing'
 'children_playing' 'children_playing' 'dog_bark' 'dog_bark' 'dog_bark'
 'air_conditioner' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'siren' 'street_music' 'street_music' 'street_music' 'street_music'
 'street_music' 'dog_bark' 'gun_shot' 'children_playing' 'siren'
 'children_playing' 'children_playing' 'street_music' 'dog_bark'
 'dog_bark' 'siren' 'siren' 'siren' 'siren' 'siren' 'siren' 'siren'
 'siren' 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'siren' 'siren' 'siren' 'siren' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'air_conditioner' 'air_conditioner' 'street_music' 'dog_bark'
 'street_music' 'street_music' 'jackhammer' 'jackhammer' 'air_conditioner'
 'air_conditioner' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'street_music' 'air_conditioner' 'street_music' 'street_music'
 'street_musi

PREDICTIONS ON FOLD 5 ['dog_bark' 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'drilling' 'children_playing'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'children_playing' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'children_playing' 'children_playing' 'drilling'
 'children_playing' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'dog_bark' 'dog_bark' 'dog_bark' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer'

PREDICTIONS ON FOLD 6 ['drilling' 'drilling' 'drilling' 'jackhammer' 'jackhammer' 'jackhammer'
 'gun_shot' 'jackhammer' 'jackhammer' 'children_playing' 'jackhammer'
 'children_playing' 'children_playing' 'drilling' 'air_conditioner'
 'air_conditioner' 'air_conditioner' 'drilling' 'drilling' 'drilling'
 'drilling' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'drilling' 'children_playing' 'children_playing'
 'children_playing' 'drilling' 'drilling' 'drilling' 'dog_bark' 'dog_bark'
 'dog_bark' 'dog_bark' 'siren' 'siren' 'siren' 'siren' 'children_playing'
 'siren' 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'children_playing' 'children_playing' 'children_playing'
 'child

PREDICTIONS ON FOLD 7 ['air_conditioner' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'air_conditioner' 'air_conditioner' 'siren' 'siren' 'siren' 'siren'
 'siren' 'siren' 'siren' 'children_playing' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'drilling' 'drilling' 'drilling' 'jackhammer'
 'engine_idling' 'air_conditioner' 'air_conditioner' 'siren' 'siren'
 'siren' 'siren' 'siren' 'dog_bark' 'dog_bark' 'siren' 'drilling'
 'drilling' 'dog_bark' 'dog_bark' 'jackhammer' 'jackhammer' 'jackhammer'
 'engine_idling' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'siren' 'siren' 'siren' 'siren' 'engine_idling' 'engine_idling'
 'air_conditioner' 'air_conditioner' 'dog_bark' 'dog_bark' 'dog_bark'
 'dog_bark' 'dog_bark' 'dog_bark' 'dog_bark' 'air_conditioner'
 'air_conditioner' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'air_condi

PREDICTIONS ON FOLD 8 ['dog_bark' 'dog_bark' 'dog_bark' 'children_playing' 'children_playing'
 'children_playing' 'car_horn' 'car_horn' 'dog_bark' 'dog_bark' 'siren'
 'air_conditioner' 'siren' 'air_conditioner' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'air_conditioner' 'air_conditioner' 'air_conditioner' 'engine_idling'
 'engine_idling' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'air_conditioner' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'air_conditioner' 'air_conditioner' 'air_conditioner' 'air_conditioner'
 'air_conditioner' 'air_condit

PREDICTIONS ON FOLD 9 ['drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'drilling' 'drilling' 'drilling'
 'drilling' 'drilling' 'drilling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'engine_idling' 'engine_idling'
 'engine_idling' 'engine_idling' 'jackhammer' 'jackhammer' 'jackhammer'
 'drilling' 'drilling' 'drilling' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer' 'jackhammer'
 'drilling' 'jackhammer' 'jackhammer' 'dr

In [ ]:
## visualization

for c in classes:
    mfcc = train_features_matrix[c][0].transpose()
    # Visualization
    fig = plt.figure(figsize=(16, 6))
    plt.subplot(1,2,1)
    plt.imshow(mfcc, origin='lower', aspect='auto')
    plt.xlabel('Training samples')
    plt.ylabel('MFCC coefficients')
    plt.title('MFCC (coefficients 0 to 13) for class {}'.format(c))
    plt.colorbar()
    plt.tight_layout()

    mfcc_upper = mfcc[4:]
    plt.subplot(1,2,2)
    plt.imshow(mfcc_upper, origin='lower', aspect='auto')
    plt.title('MFCC (coefficients 4 to 13) for class {}'.format(c))
    plt.xlabel('Training samples')
    plt.ylabel('MFCC coefficients')
    plt.colorbar()
    plt.tight_layout()